In [9]:
# Import statements needed
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch import nn
from torch import optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import os
import torchvision
import time
import pandas as pd
import random
import copy
from tqdm import tqdm 
import torchvision.models as models
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [10]:
# Use GPU if applicable
print(torch.cuda.is_available())
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
device

True


'cuda'

## Set up dataset + Helper Methods

In [11]:
# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Download the CIFAR-100 dataset (downloads only first time running cell)
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [12]:
def train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model):
    # variables needed for metrics later
    train_losses = []
    test_losses = 0
    train_accuracy = []
    test_accuracy = 0
    start_time_train = time.time()
    ############################ Train Loop ############################
    for epoch in range(epochs):
        # variables needed for metrics later
        train_size = len(train_dataloader.dataset)
        # makes sure to set model to train
        model.train()
        train_loss = 0
        train_correct = 0
        train_num_batches = len(train_dataloader)
        # Just to help with keep track of how long it taking
        train_loadbar = tqdm(train_dataloader, total=train_num_batches)
        for batch, (X, labels) in enumerate(train_loadbar):
            # Make sure values are on correct device
            X = X.to(device)
            labels = labels.to(device)

            # Model pred + loss
            pred = model(X)
            loss = loss_func(pred, labels)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Compute metrics
            train_loss+=loss.item()
            train_correct+=(pred.argmax(axis = 1) == labels).type(torch.float).sum().item()

            # Update the loading bar    
            train_loadbar.set_description(f'Epoch [{epoch + 1}/{epochs}]')
            train_loadbar.set_postfix(train_loss=train_loss/(batch + 1), train_accuracy=train_correct/train_size)


        # Compute metrics
        train_losses.append(train_loss/train_num_batches)
        train_accuracy.append(train_correct/train_size)

    end_time_train = time.time()
    train_time = end_time_train - start_time_train
    ############################ Train Loop ############################
    
    ############################ Test Loop #############################
    test_size = len(test_dataloader.dataset)
    test_num_batches = len(test_dataloader)
    # makes sure to set model to eval
    model.eval()
    # variables needed for metrics later
    start_time_test = time.time()
    test_loss = 0
    test_correct = 0
    with torch.no_grad():
        for X, labels in test_dataloader:
            # Make sure values are on correct device
            X = X.to(device)
            labels = labels.to(device)

            # Model pred + loss
            pred = model(X)
            loss = loss_func(pred, labels)

            # Compute metrics
            test_loss+=loss.item()
            test_correct+=(pred.argmax(axis = 1) == labels).type(torch.float).sum().item()
        # Compute metrics
        test_losses = test_loss/test_num_batches
        test_accuracy = test_correct/test_size
    
    end_time_test = time.time()
    test_time = end_time_test - start_time_test
    ############################ Test Loop #############################

    return train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time

## Main Training/Testing Loop (Default Hyperparams)

In [ ]:
# Compact high-level loop that runs everything and allows modification of all variables
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1', pretrained = True)
# Make 10 class classifier
model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
model.to(device)
############################################# HYPER PARAMS #############################################
batch_size = 64
loss_func = nn.CrossEntropyLoss()
lr = .0001
weight_decay = .0001
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
epochs = 30
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=epochs/3, gamma=.5)
############################################# HYPER PARAMS #############################################
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)
train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time = train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model)
# Print results to table
data = [train_time, test_time, train_accuracy[-1], test_accuracy, 64, .0001]
torch.save(model.state_dict(), f'models/Default_Model.pth')
print(data)

## Main Training/Testing Loop (Altered Hyperparams)

In [ ]:
# Compact high-level loop that runs everything and allows modification of all variables + make pd dataframe to store all information
columns = ["Train_Time", "Test_Time", "Train_Acc", "Test_Acc", "Batch", "LR"]
df = pd.DataFrame(columns = columns)
for l in [.0002, .00005]:
    for b in [16, 32, 128, 256, 512]:
        model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1', pretrained = True)
        # Make 10 class classifier
        model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
        model.to(device)
        ############################################# HYPER PARAMS #############################################
        batch_size = b
        loss_func = nn.CrossEntropyLoss()
        lr = l
        weight_decay = .0001
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        epochs = 30
        #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=epochs/3, gamma=.5)
        ############################################# HYPER PARAMS #############################################
        train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size)
        test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)
        train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time = train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model)
        # Add results to table
        data = [train_time, test_time, train_accuracy[-1], test_accuracy, b, l]
        row = pd.DataFrame([data], columns=columns)
        df = pd.concat([df, row], ignore_index=True)
        print(data)
        torch.save(model.state_dict(), f'models/model_{l}_{b}.pth')

### Save Model Results

In [138]:
df.to_csv('results.csv', index=False)

## Confusion Matrix

In [ ]:
models_directory = 'models/'
model_files = [f for f in os.listdir(models_directory) if f.endswith('.pth')]

test_dataloader = torch.utils.data.DataLoader(test, batch_size=64)
class_names = ["Airplane", "Automobile", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"]

for model_path in model_files:
    path = os.path.join(models_directory, model_path)
    model_weights = torch.load(path)

    model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1')
    model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
    model.load_state_dict(model_weights)
    model.to(device)

    predicted_labels = []
    true_labels = []

    print(path)

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            
            outputs = model(inputs.to(device))
            _, preds = torch.max(outputs, 1)
            predicted_labels.extend(preds.cpu().detach().numpy())
            true_labels.extend(labels.cpu().detach().numpy())

    cm = confusion_matrix(true_labels, predicted_labels)

    # Plot the confusion matrix using Seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names, cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
df = pd.read_csv("results.csv")

# Group data by LR (since we have 2 different LRs)
grouped = df.groupby('LR')

# Create plots for each LR
for lr, group in grouped:
    _, ax1 = plt.subplots()

    ax1.set_xlabel('Batch Size')
    # Left y-axis
    ax1.set_ylabel('Accuracy(%)', color='blue')
    ax1.plot(group['Batch'], group['Test_Acc']*100, marker='o', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Right y-axis
    ax2 = ax1.twinx()
    ax2.set_ylabel('Training Time (seconds)', color='red')
    ax2.plot(group['Batch'], group['Train_Time'], marker='o', color='red')
    ax2.tick_params(axis='y', labelcolor='red')

    x_ticks_values = [16, 32, 128, 256, 512]
    ax1.set_xticks(x_ticks_values)

    plt.title(f'Accuracy/Training Time vs. Batch Size for (LR={lr})')
plt.show()


### Generate Images

In [ ]:
# Loop through different values of size and k
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Create a mapping using the values given by CIFAR-10 website
class_mapping = [("Airplane" , 0), ("Automobile" , 1), ("Bird" , 2), ("Cat" , 3), ("Deer" , 4), ("Dog" , 5), ("Frog" , 6), ("Horse" , 7), ("Ship" , 8), ("Truck" , 9)]
# number of categories
num_categories = len(class_mapping)

# 5 images per column
fig_width = 5 * num_categories * 2
fig_height = num_categories * 2

# Generate and display 5 random images per category as one row
for class_name, label in class_mapping:
    category_images = [img for img, l in train if l == label]
    random_indices = random.sample(range(len(category_images)), 5)
    
    # Adjust figure size for displaying images
    plt.figure(figsize=(fig_width, fig_height))
    
    for i, idx in enumerate(random_indices):
        img = category_images[idx]
        # expects H x W x C images not C x H x W
        img = np.transpose(img, (1, 2, 0)) 
        plt.subplot(1, num_categories * 5, i + 1)
        plt.imshow(img)
        if i == 0:  # only for first image prevents
            plt.title(class_name)
        plt.axis('off')
    plt.show()

# Outlier Test Scenario 

In [19]:
def k_1_test(k_dataload, model):
    preds_vec = None
    preds = None
    # makes sure to set model to eval
    model.eval()
    with torch.no_grad():
        for X, labels in k_dataload:
            # Make sure values are on correct device
            X = X.to(device)

            # Model pred
            preds_vec = model(X)

            preds = preds_vec.argmax(axis = 1)

    return preds_vec, preds

In [47]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor()
])
k_folder = torchvision.datasets.ImageFolder(root='k_images', transform=transform)
k_dataload = torch.utils.data.DataLoader(k_folder, batch_size=10)


In [ ]:
# Test on default model
model_weights = torch.load("models/Default_Model.pth")
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1')
model.classifier[1] = nn.Conv2d(512, 10, kernel_size=(1, 1), stride=(1, 1))
model.load_state_dict(model_weights)
model.to(device)
preds_vec, preds = k_1_test(k_dataload, model)

print(preds_vec)
print(preds)

In [5]:
resnet50 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet18 = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
vgg11 = torchvision.models.vgg11(pretrained=True)
vgg19 = torchvision.models.vgg19(pretrained=True)
mobilenet = torchvision.models.mobilenet_v2(pretrained=True)

mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, 10)

resnet50.fc = nn.Linear(resnet50.fc.in_features, 10)
resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)

vgg11.classifier[6] = nn.Linear(vgg11.classifier[6].in_features, 10)
vgg19.classifier[6] = nn.Linear(vgg19.classifier[6].in_features, 10)
models = [resnet18, resnet50, mobilenet, vgg11, vgg19]

Using cache found in C:\Users\Shaan/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Shaan\miniconda3\envs\pytorc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shaan\miniconda3\envs\pytorc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in C:\Users\Shaan/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Shaan\miniconda3\envs\pytorc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and m

In [ ]:
# Test other models
device = 'cuda'
pred_vec_list = []
preds_list = []
for model in models:
    model.to(device)
    ############################################# HYPER PARAMS #############################################
    batch_size = 64
    loss_func = nn.CrossEntropyLoss()
    lr = .001
    weight_decay = .0001
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    epochs = 10
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=epochs/3, gamma=.5) 
    ############################################# HYPER PARAMS #############################################
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)
    train_accuracy, train_losses, test_accuracy, test_losses, train_time, test_time = train_test_loop(loss_func, optimizer, epochs, train_dataloader, test_dataloader, model)
    # Add results to table
    data = [train_time, test_time, train_accuracy[-1], test_accuracy]
    preds_vec, preds = k_1_test(k_dataload, model)
    pred_vec_list.append(preds_vec)
    preds_list.append(preds)

In [ ]:
pred_vec_list

In [ ]:
preds_list